## Predizione Risultato finale Serie A 2021/2022
Alleno un modello SVC a predire i risultati finali dei match di serie a conoscendo solo i risultati finali (1-X-1) delle altre partite

Inserisco le librerie

In [1]:
from os import error
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd

import numpy as np
from numpy.core.defchararray import add
from numpy.lib.function_base import append
import scipy as sp
from scipy.sparse import data
import sklearn as sl
from scipy import stats
from sklearn import datasets
from sklearn import linear_model

inserisco il file e carico il dataframe

In [2]:
# Load the dataset
filename = r'C:\Users\Marco\Desktop\soccer machine learning\Project1_prediction\stats_serieA_21_22.csv'
df = pd.read_csv(filename)
print("Dataset shape:", df.shape)
print("Dataset Example:")
df

Dataset shape: (180, 105)
Dataset Example:


,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,I1,21/08/2021,17:30,Inter,Genoa,4,0,H,2,0,...,2.32,-1.50,1.88,2.05,1.89,2.03,1.96,2.09,1.86,2.01
1,I1,21/08/2021,17:30,Verona,Sassuolo,2,3,A,0,1,...,2.03,0.00,1.88,2.05,1.89,2.04,2.00,2.07,1.86,2.02
2,I1,21/08/2021,19:45,Empoli,Lazio,1,3,A,1,3,...,2.21,0.50,2.00,1.93,2.00,1.93,2.06,1.95,1.99,1.88
3,I1,21/08/2021,19:45,Torino,Atalanta,1,2,A,0,1,...,2.45,1.00,2.06,1.87,2.08,1.85,2.19,1.90,2.03,1.84
4,I1,22/08/2021,17:30,Bologna,Salernitana,3,2,H,0,0,...,2.16,-1.00,1.94,1.99,1.92,2.01,2.02,2.01,1.91,1.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,I1,19/12/2021,11:30,Fiorentina,Sassuolo,2,2,D,0,2,...,2.55,-0.50,1.99,1.94,1.97,1.95,2.00,2.01,1.94,1.94
176,I1,19/12/2021,14:00,Spezia,Empoli,1,1,D,0,0,...,2.24,0.00,1.96,1.97,1.98,1.94,2.10,2.04,1.96,1.92
177,I1,19/12/2021,17:00,Sampdoria,Venezia,1,1,D,1,0,...,1.91,-0.75,2.06,1.87,2.06,1.87,2.09,1.89,2.04,1.84
178,I1,19/12/2021,17:00,Torino,Verona,1,0,H,1,0,...,1.89,-0.25,1.96,1.97,1.94,1.98,2.10,2.00,1.93,1.95


estraggo Y e X ovvero label e features

In [3]:
# creo le labels y, utilizzando 1-2-3 (H-D-A)
y = np.array(df['FTR'])
for i in range(len(y)):
    if y[i] == 'H':
        y[i] = 1
    if y[i] == 'D':
        y[i] = 2
    if y[i] == 'A':
        y[i] = 3
print(y)

[1 3 3 3 1 2 1 1 2 3 1 3 2 1 1 3 3 2 1 3 3 1 3 2 3 3 1 1 1 1 3 3 1 3 3 3 2
 1 2 3 2 1 3 2 3 3 1 3 2 1 3 2 2 1 1 1 3 1 1 2 2 1 3 3 1 2 1 3 1 3 1 1 1 1
 3 2 2 1 1 1 1 1 3 1 3 2 1 1 2 2 2 3 1 3 3 2 3 3 1 1 2 1 1 1 1 2 3 3 3 1 2
 1 1 3 1 3 1 1 2 2 1 3 1 2 3 3 1 3 1 1 2 1 1 3 3 2 3 3 1 1 1 1 3 2 1 1 3 2
 2 2 1 3 3 3 2 3 3 1 1 2 3 1 2 2 1 3 3 1 1 1 1 3 3 3 3 2 2 2 1 3]


In [10]:
# creo X, le features con la 'dummy selection'
X_prov = df.iloc[:, 3:5] # X provvisorio
team_names = np.sort(df['HomeTeam'].unique()) # elenco nomi squadre
for name in team_names:
    nome = str(name)
    X_prov[nome + '_home'] = np.where(df['HomeTeam'] == nome, 1, 0)
    X_prov[nome + '_away'] = np.where(df['AwayTeam'] == nome, 1, 0)
X = X_prov.drop(columns=['HomeTeam', 'AwayTeam'])
X

,Atalanta_home,Atalanta_away,Bologna_home,Bologna_away,Cagliari_home,Cagliari_away,Empoli_home,Empoli_away,Fiorentina_home,Fiorentina_away,...,Spezia_home,Spezia_away,Torino_home,Torino_away,Udinese_home,Udinese_away,Venezia_home,Venezia_away,Verona_home,Verona_away
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
176,0,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
177,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
178,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1


In [11]:
# split in test e training
from sklearn.model_selection import train_test_split as ttsplit
TestSize = 0.25
X_train, X_test, y_train, y_test = ttsplit(X, y, test_size=TestSize, shuffle=False)

Adesso utilizzo una SVM con kernel polinomiale (degree=3)

In [16]:
#import SVC
from sklearn.svm import SVC
#import for Cross-Validation
from sklearn.model_selection import GridSearchCV

C_values = gamma_values = list(np.linspace(0.01, 10, 10))
parameters = {'C': C_values, 'gamma': gamma_values}

#train linear SVM
clf = GridSearchCV(SVC(kernel='poly', degree=3), parameters, cv=2)
y_train = y_train.astype('int')
clf.fit(X_train, y_train)
print("best parameters found ", clf.best_params_)

best parameters found  {'C': 0.01, 'gamma': 3.34}


In [19]:
# stampo un po' di risultati
y_pred_svc = clf.predict(X_test)
print("y_pred_svc:", y_pred_svc)
print("y_test:    ", y_test)
print("SVC train score:", clf.score(X_train, y_train.astype(int))*100, "%")
print("SVC test score:", clf.score(X_test, y_test.astype(int))*100, "%")

y_pred_svc: [2 1 1 1 1 1 1 3 1 1 1 3 3 1 1 1 3 1 1 3 1 1 1 3 3 3 1 3 3 1 3 1 1 1 1 1 3
 1 3 2 1 3 1 1 1]
y_test:     [2 3 3 1 1 1 1 3 2 1 1 3 2 2 2 1 3 3 3 2 3 3 1 1 2 3 1 2 2 1 3 3 1 1 1 1 3
 3 3 3 2 2 2 1 3]
SVC train score: 100.0 %
SVC test score: 51.11111111111111 %


In [18]:
# facciamo qualche predizione
#  segna 1 sotto una squadra se gioca: sx se gioca in casa, dx in trasferta
#                 ata   bol cag  emp  fio  gen  int  juv  laz  mil  nap  rom  sal  sam  sas  spe  tor  udi  ven  ver
p = clf.predict([[0,0, 0,0, 0,0, 0,0, 0,0, 0,0, 1,0, 0,0, 0,0, 0,0, 0,0, 0,1, 0,0, 0,0, 0,0, 0,0, 0,0, 0,0, 0,0, 0,0]])
print(p)

[1]
